In [1]:
import os
import pandas as pd
import numpy as np
import scipy
from itertools import product

# Define the folder containing the CSV files
folder_path = 'results_knn'

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if 'sick' in f]
# Initialize an empty list to store dataframes
df_list = []
# Loop through each file, read it, and append the dataframe to the list
for file in csv_files:
    file_path = os.path.join(folder_path, file)  # Full file path
    df = pd.read_csv(file_path)  # Read the CSV file
    df_list.append(df)  # Add the dataframe to the list
# Concatenate all dataframes into a single dataframe
df_combined = pd.concat(df_list, ignore_index=True)
# Now df_combined has all the data from the 10 folds
print(df_combined.head())


   K    Distance   Voting scheme      Weight scheme  Accuracy  Precision  \
0  1  minkowski1  Majority_class  Mutual_classifier  0.941489   0.933891   
1  3  minkowski1  Majority_class  Mutual_classifier  0.957447   0.942690   
2  5  minkowski1  Majority_class  Mutual_classifier  0.957447   0.940323   
3  7  minkowski1  Majority_class  Mutual_classifier  0.957447   0.940323   
4  1  minkowski1  Majority_class             Relief  0.957447   0.916704   

     Recall        F1  Solving Time  
0  0.941489  0.937472      7.495436  
1  0.957447  0.945030      8.413954  
2  0.957447  0.941304      8.346267  
3  0.957447  0.941304      8.325003  
4  0.957447  0.936633     14.394366  


In [11]:
grouped_df = df_combined.groupby(['K', 'Distance', 'Voting scheme', 'Weight scheme'])

# Compute mean and standard deviation of the relevant metrics
"""
metrics_summary = grouped_df.agg({
    'Accuracy': ['mean', 'std'],
    'Precision_Class_0': ['mean', 'std'],
    'Recall_Class_0': ['mean', 'std'],
    'F1_Class_0': ['mean', 'std'],
    'Precision_Class_1': ['mean', 'std'],
    'Recall_Class_1': ['mean', 'std'],
    'F1_Class_1': ['mean', 'std'],
    'Solving Time': ['mean', 'std']
}).reset_index()"""

metrics_summary = grouped_df.agg({
    'Accuracy': ['mean', 'std'],
    'Precision': ['mean', 'std'],
    'Recall': ['mean', 'std'],
    'F1': ['mean', 'std'],
    'Solving Time': ['mean', 'std']
}).reset_index()

# Rename the columns for clarity
metrics_summary.columns = ['K', 'Distance', 'Voting scheme', 'Weight scheme',
                           'Accuracy_mean', 'Accuracy_std',
                           'Precision_mean', 'Precision_std',
                           'Recall_mean', 'Recall_std',
                           'F1_mean', 'F1_std',
                           'Solving Time_mean', 'Solving Time_std']

best_models = metrics_summary.sort_values(by='Accuracy_mean', ascending = False).head(5)

In [12]:
best_models[['K', 'Distance', 'Voting']]

,K,Distance,Voting scheme,Weight scheme,Accuracy_mean,Accuracy_std,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,Solving Time_mean,Solving Time_std
93,7,minkowski1,Majority_class,ANOVA,0.962347,0.008250,0.949604,0.016610,0.962347,0.008250,0.950174,0.011148,7.332197,0.941769
84,7,HEOM,Majority_class,ANOVA,0.961818,0.008564,0.949160,0.016742,0.961818,0.008564,0.949560,0.011719,24.507832,2.188125
102,7,minkowski2,Majority_class,ANOVA,0.961818,0.008564,0.949160,0.016742,0.961818,0.008564,0.949560,0.011719,12.443318,1.417802
103,7,minkowski2,Majority_class,Mutual_classifier,0.961818,0.009430,0.951001,0.017723,0.961818,0.009430,0.948998,0.012768,13.142803,1.501413
96,7,minkowski1,Sheppards_Work,ANOVA,0.961817,0.008382,0.948629,0.016234,0.961817,0.008382,0.949209,0.011231,6.945436,0.793663


In [13]:
# Choose the metric you want to analyze (e.g., Accuracy)
metric = 'Accuracy'

# Generate the unique values for each column
voting_schemes = df_combined['Voting scheme'].unique()
weight_schemes = df_combined['Weight scheme'].unique()
distances = df_combined['Distance'].unique()
k_values = df_combined['K'].unique()

# Create all combinations of the unique values
combinations = list(product(voting_schemes, weight_schemes, distances, k_values))

results = []
metric_values = []

for i, row in best_models.iterrows():  
    
    # Filter the DataFrame for the current hyperparameter combination
    filtered_df = df_combined[(df_combined['Voting scheme'] == row['Voting scheme']) & 
                             (df_combined['Weight scheme'] == row['Weight scheme']) &
                             (df_combined['Distance'] == row['Distance']) &
                             (df_combined['K'] == row['K'])]
    
    metric_values.append(list(filtered_df[metric].values))

# Perform the Friedman test
stat, p_value = scipy.stats.friedmanchisquare(*metric_values)

print(f'Friedman test statistic: {stat}, p-value: {p_value}')

# Interpret the results_knn
alpha = 0.05  # significance level
if p_value < alpha:
    print("There are significant differences between the hyperparameter combinations.")
else:
    print("No significant differences found between the hyperparameter combinations.")

Friedman test statistic: 2.3098591549295584, p-value: 0.6789746183067471
No significant differences found between the hyperparameter combinations.
